# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Defining the Problem

Create a supervised machine learning model to predict the price of a used car based on its attributes, such as mileage, age, drivetrain, engine type, and brand.

Use statistical and machine learning techniques to determine the relative importance of each feature in influencing the price.

### Terms

Technical Vocabulary:

Target Variable: The sale price of the car (dependent variable).
Features: Car attributes such as mileage, age, cylinder count, fuel type, transmission, drivetrain, brand, and condition.
Task Type: Regression (predicting a continuous variable) combined with feature importance analysis to interpret the model.
Modeling Goals:
Develop a regression model that minimizes error (e.g., Mean Squared Error).
Identify features with the highest coefficients or feature importance scores to provide actionable insights.

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn import set_config
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import psutil

# Set configuration for pipeline diagram display
set_config(display="diagram")

In [ ]:
df = pd.read_csv('data/vehicles.csv')

In [ ]:
# Display the first few rows of the dataframe to verify
df.describe()

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Check the size of the dataset before any processing
print("Initial size of the dataset:", df.shape)

In [ ]:
# Unique values and their frequencies in the 'cylinders' column:
# print(df['cylinders'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'fuel' column:
# print(df['fuel'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'transmission' column:
# print(df['transmission'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'drive' column:
# print(df['drive'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'size' column:
# print(df['size'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'type' column:
# print(df['type'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'manufacturer' column:
# print(df['manufacturer'].value_counts())

In [ ]:
# Unique values and their frequencies in the 'state' column:
# print(df['state'].value_counts())

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

In [ ]:
# Remove duplicates
df = df.drop_duplicates()
print("Size of the dataset after removing duplicates:", df.shape)

# Remove rows where 'price' is 0, negative, blank, or above 200,000
df = df[(df['price'] > 0) & (df['price'] <= 200000)]  
print(f"Dataset shape after filtering: {df.shape}")


In [ ]:
# Create New Features

# Custom Feature 1 - Age of the Car
current_year = pd.Timestamp.now().year
df['car_age'] = current_year - df['year']

# Custom Feature 2 - Luxury Indicator
luxury_brands = [
    'bmw', 
    'mercedes-benz', 
    'lexus', 
    'audi', 
    'cadillac', 
    'acura', 
    'infiniti', 
    'lincoln', 
    'volvo', 
    'porsche', 
    'jaguar', 
    'land rover', 
    'alfa-romeo', 
    'ferrari', 
    'aston-martin'
]

df['is_luxury'] = df['manufacturer'].apply(lambda x: 1 if x in luxury_brands else 0)

# 4. Vehicle Size and Type
#df['size'] = df['size'].fillna('unknown')  # Handle missing values
#df['type'] = df['type'].fillna('unknown')

# Custom Feature 3 - Condition Index
condition_mapping = {
    'new': 5,
    'like new': 4,
    'excellent': 3,
    'good': 2,
    'fair': 1,
    'salvage': 0
}
df['condition_index'] = df['condition'].map(condition_mapping)

# Custom Feature 4 - eco (hybrid or electric)
df['eco'] = df['fuel'].apply(lambda x: 1 if x in ['hybrid', 'electric'] else 0)

# Custom Feature 5 - Paint Popularity
popular_colors = ['black', 'white', 'silver', 'gray', 'blue']
df['is_popular_color'] = df['paint_color'].apply(lambda x: 1 if x in popular_colors else 0)

# Custom Feature 6 - Drive Code
df['drive_code'] = df['drive'].apply(lambda x: 1 if x == '4wd' else 0)

# Custom Feature 7 - Title Code
df['title_code'] = df['title_status'].apply(lambda x: 1 if x == 'clean' else 0)

# Custom Feature 8 - Transmission Code
df['transmission_code'] = df['transmission'].apply(lambda x: 1 if x == 'automatic' else 0)

# Custom Feature 9 - Cylinder Code
df = df[df['cylinders'].notna() & (df['cylinders'] != 'other')]

# Extract the numerical part from the 'cylinders' column and create a new column 'cylinder_code'
df['cylinder_code'] = df['cylinders'].str.extract(r'(\d+)')  # Use raw string for regex

# Drop rows where extraction failed (results in NaN) and convert to integer
df = df[df['cylinder_code'].notna()]
df['cylinder_code'] = df['cylinder_code'].astype(int)

# Custom Feature 10 - Group States by Geographic Region
state_to_region = {
    'AL': 'South', 'AK': 'West', 'AZ': 'West', 'AR': 'South', 'CA': 'West',
    'CO': 'West', 'CT': 'Northeast', 'DE': 'South', 'FL': 'South', 'GA': 'South',
    'HI': 'West', 'ID': 'West', 'IL': 'Midwest', 'IN': 'Midwest', 'IA': 'Midwest',
    'KS': 'Midwest', 'KY': 'South', 'LA': 'South', 'ME': 'Northeast', 'MD': 'South',
    'MA': 'Northeast', 'MI': 'Midwest', 'MN': 'Midwest', 'MS': 'South', 'MO': 'Midwest',
    'MT': 'West', 'NE': 'Midwest', 'NV': 'West', 'NH': 'Northeast', 'NJ': 'Northeast',
    'NM': 'West', 'NY': 'Northeast', 'NC': 'South', 'ND': 'Midwest', 'OH': 'Midwest',
    'OK': 'South', 'OR': 'West', 'PA': 'Northeast', 'RI': 'Northeast', 'SC': 'South',
    'SD': 'Midwest', 'TN': 'South', 'TX': 'South', 'UT': 'West', 'VT': 'Northeast',
    'VA': 'South', 'WA': 'West', 'WV': 'South', 'WI': 'Midwest', 'WY': 'West',
    'DC': 'South'  # Adding DC as part of the South region
}

# Convert all entries in the 'state' column to uppercase
df['state'] = df['state'].str.upper()

# Reapply the mapping after converting to uppercase
df['state_region'] = df['state'].map(state_to_region)

# Assign a unique code to each region
region_mapping = {region: idx + 1 for idx, region in enumerate(df['state_region'].unique())}

# Map the region codes to a new column
df['region_code'] = df['state_region'].map(region_mapping)

# Handling Missing Values
df['car_age'] = df['car_age'].fillna(df['car_age'].median())
df['condition_index'] = df['condition_index'].fillna(0)  
df['drive'] = df['drive'].fillna('unknown')
df['transmission'] = df['transmission'].fillna('unknown')
df['fuel'] = df['fuel'].fillna('unknown') 

# Drop rows with odometer > 500000 or car_age > 100
df = df[(df['odometer'] <= 500000) & (df['car_age'] <= 100)]
df.drop(columns=['id', 'VIN', 'paint_color', 'region', 'condition', 'model'], inplace=True)

# Check the new dataset shape
print(f"Dataset shape after filtering: {df.shape}")

print("Data preparation complete. Ready for preprocessing.")


### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

In [ ]:
# Get total and available memory
total_memory = psutil.virtual_memory().total / (1024 ** 3)  # Convert to GB
available_memory = psutil.virtual_memory().available / (1024 ** 3)  # Convert to GB

print(f"Total Memory: {total_memory:.2f} GB")
print(f"Available Memory: {available_memory:.2f} GB")


In [ ]:
# Select features and target
features = ['cylinder_code', 'eco', 'odometer', 'title_code', 
            'region_code', 'car_age', 'is_luxury', 
            'condition_index', 'is_popular_color', 'transmission_code', 'drive_code']

X = df[features].copy()

# Log-transform the target
df['price_log'] = np.log1p(df['price'])

y = df['price_log']

# Separate numerical and categorical columns
numerical_columns = X.select_dtypes(include=['number']).columns

# Impute missing values
num_imputer = SimpleImputer(strategy='median')

X[numerical_columns] = num_imputer.fit_transform(X[numerical_columns])

# One-hot encode categorical features
# X = pd.get_dummies(X, drop_first=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Scale the target variable
#y_scaler = StandardScaler()
#y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
#y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).ravel()

model_selector_pipe = Pipeline([
    ('poly_features', PolynomialFeatures(degree=3, include_bias=False)),  # Adjust degree if necessary
    ('scaler', StandardScaler()),
    ('selector', SelectFromModel(Lasso(alpha=0.05, max_iter=3000))),  # Increased alpha and max_iter
    ('linreg', LinearRegression())
])

model_selector_pipe.fit(X_train, y_train)

# Evaluate
selector_train_mse = mean_squared_error(y_train, model_selector_pipe.predict(X_train))
selector_test_mse = mean_squared_error(y_test, model_selector_pipe.predict(X_test))

# Convert predictions back to original scale
#predictions_original_scale = y_scaler.inverse_transform(model_selector_pipe.predict(X_test).reshape(-1, 1)).ravel()

# Results
print("Train MSE:", selector_train_mse)
print("Test MSE:", selector_test_mse)


In [ ]:
# Compute MSE on the log-transformed (scaled) target
lasso_train_mse_scaled = mean_squared_error(y_train, model_selector_pipe.predict(X_train))
lasso_test_mse_scaled = mean_squared_error(y_test, model_selector_pipe.predict(X_test))

# Convert predictions back to the original scale (reverse log transformation)
train_predictions_original = np.expm1(model_selector_pipe.predict(X_train))
test_predictions_original = np.expm1(model_selector_pipe.predict(X_test))

# Convert actual values back to the original scale (reverse log transformation)
y_train_original = np.expm1(y_train)
y_test_original = np.expm1(y_test)

# Compute MSE on the original scale
lasso_train_mse = mean_squared_error(y_train_original, train_predictions_original)
lasso_test_mse = mean_squared_error(y_test_original, test_predictions_original)

# Results
print("Train MSE (Log-Transformed Scale):", lasso_train_mse_scaled)
print("Test MSE (Log-Transformed Scale):", lasso_test_mse_scaled)
print("Train MSE (Original Scale):", lasso_train_mse)
print("Test MSE (Original Scale):", lasso_test_mse)



In [ ]:
# Extract feature names after polynomial feature expansion
feature_names = model_selector_pipe.named_steps['poly_features'].get_feature_names_out(X_train.columns)

# Extract coefficients from the Lasso step
lasso_coefs = model_selector_pipe.named_steps['selector'].estimator_.coef_

# Create a DataFrame to display features and their corresponding coefficients
lasso_df = pd.DataFrame({'feature': feature_names, 'coef': lasso_coefs})

# Sort the DataFrame by the absolute value of coefficients for better insight
lasso_df['abs_coef'] = lasso_df['coef'].abs()
lasso_df = lasso_df.sort_values(by='abs_coef', ascending=False)

# Results
print(type(feature_names))
print(lasso_df.loc[lasso_df['coef'] != 0])  




### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Top 20 Features by Coefficient Value (Log-Transformed Target)
sorted_lasso_df = lasso_df.head(20) 

# Create bar plot
plt.figure(figsize=(12, 8))
plt.barh(
    sorted_lasso_df['feature'],
    sorted_lasso_df['coef'],
    color=['red' if c < 0 else 'blue' for c in sorted_lasso_df['coef']]
)
plt.xlabel('Coefficient Value (Log-Transformed Target)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Top 20 Factors by Coefficient Value', fontsize=14)
plt.gca().invert_yaxis()  # Invert y-axis for better readability
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()



In [ ]:
# Predictions on log-transformed scale
y_train_pred_log = model_selector_pipe.predict(X_train)
y_test_pred_log = model_selector_pipe.predict(X_test)

# Convert predictions back to the original scale
y_train_pred_original = np.expm1(y_train_pred_log)
y_test_pred_original = np.expm1(y_test_pred_log)

# Convert actual values back to the original scale
y_train_original = np.expm1(y_train)
y_test_original = np.expm1(y_test)

# Residuals on the original scale
residuals_test = y_test_original - y_test_pred_original

# Residual plot for test set
plt.figure(figsize=(8, 6))
plt.scatter(y_test_original, residuals_test, alpha=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual Plot (Original Scale)', fontsize=14)
plt.xlabel('Actual Price', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.grid(alpha=0.5, linestyle='--')
plt.tight_layout()
plt.show()


In [ ]:
# Create a heatmap of coefficients
plt.figure(figsize=(10, 6))
sns.heatmap(pd.DataFrame(lasso_df.set_index('feature')['coef']).T, cmap='coolwarm', annot=True, fmt=".2f", cbar=True)
plt.title('Feature Coefficients Heatmap')
plt.show()


In [ ]:
key_features = ['odometer', 'car_age', 'price']
sns.pairplot(df[key_features])
plt.show()


### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.

### Key Findings and Suggestions:
1.	Cars with larger engines (higher cylinder counts i.e. ‘cylinder_code’) significantly increase the sale price. 
•	Suggestion: Include vehicles with 6, 8, or more cylinders in your inventory, as they are highly valued by buyers, especially for performance or utility purposes.
2.	Higher odometer readings (mileage) combined with the car’s age have a strong negative impact on price (‘odometer car_age’). Higher mileage is consistently associated with a decrease in price (‘odometer’).
•	Suggestion: Prioritize acquiring cars with low mileage, especially for older models, to maintain higher resale values.
3.	Older cars see a notable decline in value, even without factoring in mileage or other features (‘car_age’).
•	Suggestion: Focus on stocking newer models whenever possible. Highlight the condition and features of older vehicles to mitigate price reductions.
4.	Certain combinations of transmission type and car age positively influence price (‘car_age transmission_code’). Vehicles with popular colors, specific transmission types, and drivetrain configurations tend to attract higher prices (is_popular_color transmission_code drive_code’).
•	Suggestion: Acquire cars with automatic transmissions, especially those paired with desirable colors or AWD/4WD configurations.
5.	Popular exterior colors marginally increase sale price (‘is_popular_color’). Luxury vehicles or configurations positively influence pricing (‘is_luxury’)
•	Suggestion: Stock luxury brands and highlight popular colors to attract buyers willing to pay a premium.

